In [22]:
import numpy as np
import random
import time
from IPython.display import display,Latex,Math
%matplotlib inline
 
from IPython.core.interactiveshell import InteractiveShell
sh = InteractiveShell.instance()
 
def number_to_str(n,cut=5):
    ns=str(n)
    format_='{0:.'+str(cut)+'f}'
    if 'e' in ns or ('.' in ns and len(ns)>cut+1):
        return format_.format(n)
    else:
        if n == 0:
            return ("{\color{blue} %s}" %n)
        return str(n)
 
def matrix_to_latex(mat,style='bmatrix'):
    if type(mat)==np.matrixlib.defmatrix.matrix:
        mat=mat.A
    head=r'\begin{'+style+'}'
    tail=r'\end{'+style+'}'
    if len(mat.shape)==1:
        body=r'\\'.join([str(el) for el in mat])
        return head+body+tail
    elif len(mat.shape)==2:
        lines=[]
        for row in mat:
            lines.append('&'.join([number_to_str(el)  for el in row])+r'\\')
        s=head+' '.join(lines)+tail
        return s
    return None
 
sh.display_formatter.formatters['text/latex'].type_printers[np.ndarray]=matrix_to_latex

In [24]:

class MyLinkSearch():
    def __init__(self, map_=None, auto_grader=None):
        self.auto_grader = auto_grader
#         self.pretreat(map_)

    def link(self, x1, y1, x2, y2):
            print("Linked (%s, %s) to (%s, %s)." % (x1, y1, x2, y2))
            if self.auto_grader:
                ret = self.auto_grader.link(x1-1, y1-1, x2-1, y2-1)
                self.map[x1, y1] = self.map[x2, y2] = 0
                if ret > 0:
                    return ret
                else:
                    raise Exception("ret == %s" %ret)

    def direct(self, x1, y1, x2, y2):
        if x1 == x2 and y1 == y2:
            return False
        elif x1 == x2:    # 平行y轴
            return not self.map[x1, y1:y2:(2*(y1 < y2)-1)][1:].any()
        elif y1 == y2:    # 平行x轴
            return not self.map[x1:x2:(2*(x1 < x2)-1), y1][1:].any()
        return False

    def is_empty(self, x, y):
        return not self.map[x, y]

    def get_boarder(self, x, y, direction):
        if direction=='x':
            a = b = x
            while a>0 and self.is_empty(a-1, y):
                a -= 1
            while b<9 and self.is_empty(b+1, y):
                b += 1
        elif direction=='y':
            a = b = y
            while a>0 and self.is_empty(x, a-1):
                a -= 1
            while b<9 and self.is_empty(x, b+1):
                b += 1
        return a, b + 1

    def one_corner(self, x1, y1, x2, y2):
        p1 = self.direct(x1, y2, x1, y1) and self.direct(x1, y2, x2, y2) and not self.map[x1, y2] # x1, y2
        p2 = self.direct(x2, y1, x1, y1) and self.direct(x2, y1, x2, y2) and not self.map[x2, y1] # x2, y1
        return p1 or p2
        
    def two_corner(self, x1, y1, x2, y2):
        a1, b1 = self.get_boarder(x1, y1, 'y')
        a2, b2 = self.get_boarder(x2, y2, 'y')
        for y in range(max(a1, a2), min(b1, b2)):
            if self.direct(x1, y, x2, y):
                return True
        
        a1, b1 = self.get_boarder(x1, y1, 'x')
        a2, b2 = self.get_boarder(x2, y2, 'x')
        for x in range(max(a1, a2), min(b1, b2)):
            if self.direct(x, y1, x, y2):
                return True

        return False

    def three_corner(self, x1, y1, x2, y2):
        ax1, bx1 = self.get_boarder(x1, y1, 'x')
        ay2, by2 = self.get_boarder(x2, y2, 'y')
        ax2, bx2 = self.get_boarder(x2, y2, 'x')
        ay1, by1 = self.get_boarder(x1, y1, 'y')
        
        for x in range(ax1, bx1):
            for y in range(ay2, by2):
                if self.is_empty(x, y) and self.direct(x2, y, x, y) and self.direct(x, y, x, y1):
                    print(x, y, 'a')
                    return True

        for x in range(ax2, bx2):
            for y in range(ay1, by1):
                if self.is_empty(x, y) and self.direct(x1, y, x, y) and self.direct(x, y, x, y2):
                    print(x, y, 'b')
                    return True
        
        return False

    def search_link(self, x1, y1, x2, y2):
        if x1 == x2 and y1 == y2:
            return None
        if self.direct(x1, y1, x2, y2):
            print('1 line.')
            return 50
        elif self.one_corner(x1, y1, x2, y2):
            print('2 lines.')
            return 20
        elif self.two_corner(x1, y1, x2, y2):
            print('3 lines.')
            return 10
        elif self.three_corner(x1, y1, x2, y2):
            print('4 lines.')
            return 0
        else:
            print('more than 4 lines.')
            return None
    
    def pretreat(self, map_):
        self.map = np.pad(map_,((1,1),(1,1)),'constant',constant_values = (0,0)) 


In [25]:
map_ = np.array(
    [[0,0,0,0,0,0,0,0,0,0]
    ,[0,22,7,0,20,0,0,0,16,0]
    ,[0,0,0,15,18,0,0,27,28,0]
    ,[0,2,4,4,26,0,21,23,26,0]
    ,[0,10,15,7,22,14,20,26,14,0]
    ,[0,0,14,15,14,4,23,14,14,0]
    ,[0,10,20,21,16,14,23,20,17,0]
    ,[0,2,26,15,28,23,10,17,10,0]
    ,[0,4,14,27,27,29,27,29,18,0]
    ,[0,0,0,0,0,0,0,0,0,0]])
map_

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 22,  7,  0, 20,  0,  0,  0, 16,  0],
       [ 0,  0,  0, 15, 18,  0,  0, 27, 28,  0],
       [ 0,  2,  4,  4, 26,  0, 21, 23, 26,  0],
       [ 0, 10, 15,  7, 22, 14, 20, 26, 14,  0],
       [ 0,  0, 14, 15, 14,  4, 23, 14, 14,  0],
       [ 0, 10, 20, 21, 16, 14, 23, 20, 17,  0],
       [ 0,  2, 26, 15, 28, 23, 10, 17, 10,  0],
       [ 0,  4, 14, 27, 27, 29, 27, 29, 18,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

In [26]:
ls = MyLinkSearch()
ls.map = map_

In [27]:
"""Linked (2, 7) 27 to (8, 3) 27."""
ls.search_link(2, 7, 8, 3)

more than 4 lines.
